# Model Training

1. Retrieve all feature groups
2. Join features
3. Create a model per resort location (filter by location + sort)
4. Train XGBoost + validation
5. Check performance

In [20]:
import xgboost
import pandas as pd
import datetime
import hopsworks
import warnings
import matplotlib.pyplot as plt
import os
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit
from scipy.stats import uniform, randint
from locations import *
warnings.filterwarnings("ignore", module="IPython")


In [21]:
project = hopsworks.login(
    host="eu-west.cloud.hopsworks.ai",             # DNS of your Hopsworks instance
    project="ID2223_Project"
)

fs = project.get_feature_store()

2025-12-30 13:24:00,077 INFO: Closing external client and cleaning up certificates.
2025-12-30 13:24:00,083 INFO: Connection closed.
2025-12-30 13:24:00,089 INFO: Initializing external client
2025-12-30 13:24:00,090 INFO: Base URL: https://eu-west.cloud.hopsworks.ai:443
2025-12-30 13:24:01,171 INFO: Python Engine initialized.

Logged in to project, explore it here https://eu-west.cloud.hopsworks.ai:443/p/2173


## Create Feature View

In [22]:
warning_fg = fs.get_feature_group(
    name='avalanche_warning',
    version=4
)

weather_fg = fs.get_feature_group(
    name="weather_sensor",
    version=3
)

terrain_fg = fs.get_feature_group(
    name="terrain_data",
    version=2
)

In [ ]:
selected_features = (
    warning_fg
        .select(["location", "date", "warning_level"])
        .join(
            weather_fg.select([
                "temperature_2m_mean",
                "precipitation_sum",
                "rain_sum",
                "snowfall_sum",
                "wind_speed_10m_max",
                "wind_direction_10m_dominant"]),
            on=["location", "date"]
        )
)
selected_features.features


[Feature('location', None, None, False, False, False, None, None, None),
 Feature('date', None, None, False, False, False, None, None, None),
 Feature('warning_level', None, None, False, False, False, None, None, None),
 Feature('temperature_2m_mean', None, None, False, False, False, None, None, None),
 Feature('precipitation_sum', None, None, False, False, False, None, None, None),
 Feature('rain_sum', None, None, False, False, False, None, None, None),
 Feature('snowfall_sum', None, None, False, False, False, None, None, None),
 Feature('wind_speed_10m_max', None, None, False, False, False, None, None, None),
 Feature('wind_direction_10m_dominant', None, None, False, False, False, None, None, None)]

In [25]:
feature_view = fs.create_feature_view(
    name="avalanche_warning_fv_new_corrected",
    version=1,
    description="Feature view combining avalanche warnings, weather forecasts, and static terrain features for Norwegian ski resorts.",
    query=selected_features,
    labels=["warning_level"]
)

Feature view created successfully, explore it at 
https://eu-west.cloud.hopsworks.ai:443/p/2173/fs/2122/fv/avalanche_warning_fv_new_corrected/version/1


## Train-test splits

In [26]:
start_date_test_data = "2025-06-30"
# Convert string to datetime object
test_start = datetime.datetime.strptime(start_date_test_data, "%Y-%m-%d")
X_train, X_test, y_train, y_test = feature_view.train_test_split(
    test_start=test_start
)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (2.70s) 
2025-12-30 13:25:11,075 INFO: Computing insert statistics
2025-12-30 13:25:11,469 INFO: Computing insert statistics
2025-12-30 13:25:12,729 WARNING: VersionWarning: Incremented version to `1`.



In [ ]:
print(len(X_train[X_train['location'] == 'Galdhøpiggen Summer Ski Centre'].sort_values(by='date')))
X_train[X_train['location'] == 'Galdhøpiggen Summer Ski Centre']

1643


In [30]:
def prepare_data(X, y_enc, location):
    df = (
        X.assign(label=y_enc)
        .loc[X["location"] == location]
        .sort_values("date")
        .reset_index(drop=True)
    )

    X_loc = df.drop(columns=["location", "date", "label"])
    y_loc = df["label"].to_numpy()

    return X_loc, y_loc

In [31]:
if os.path.exists("models") == False:
    os.mkdir("models")

label_encoder = LabelEncoder()
y_train_enc = label_encoder.fit_transform(y_train.to_numpy().ravel())
y_test_enc  = label_encoder.transform(y_test.to_numpy().ravel())

param_distributions = {
    "n_estimators": randint(200, 1200),
    "max_depth": randint(3, 12),
    "learning_rate": uniform(0.01, 0.2),
    "subsample": uniform(0.5, 0.5),
    "colsample_bytree": uniform(0.5, 0.5),
    "gamma": uniform(0, 5)
}

for loc in resorts.keys():
    
    print(f"Training and evaluating model for location: {loc}")

    X_train_loc, y_train_loc = prepare_data(X_train, y_train_enc, loc)
    X_test_loc,  y_test_loc  = prepare_data(X_test,  y_test_enc,  loc)

    tscv = TimeSeriesSplit(n_splits=5)

    model = xgboost.XGBClassifier(
        objective="multi:softprob",
        num_class=len(label_encoder.classes_),
        eval_metric="mlogloss",
        random_state=42
   )

    random_search = RandomizedSearchCV(
        estimator=model,
        param_distributions=param_distributions,
        n_iter=50,
        scoring="accuracy",
        cv=tscv,
        verbose=2,
        n_jobs=-1,
        random_state=42
    )

    random_search.fit(X_train_loc, y_train_loc)
    best_model = random_search.best_estimator_
    
    best_model.save_model(f"models/xgb_model_{loc.replace(' ', '_')}.json")
    
    y_pred_enc = best_model.predict(X_test_loc)

    print(f"Accuracy Score: {(accuracy_score(y_test_loc, y_pred_enc)):.3f}")
       


Training and evaluating model for location: Narvik Ski Resort
Fitting 5 folds for each of 50 candidates, totalling 250 fits
Accuracy Score: 0.707
Training and evaluating model for location: Strandafjellet Skisenter
Fitting 5 folds for each of 50 candidates, totalling 250 fits
Accuracy Score: 0.723
Training and evaluating model for location: Voss Resort Fjellheisar
Fitting 5 folds for each of 50 candidates, totalling 250 fits
Accuracy Score: 0.696
Training and evaluating model for location: Myrkdalen Fjellandsby
Fitting 5 folds for each of 50 candidates, totalling 250 fits
Accuracy Score: 0.701
Training and evaluating model for location: Nedre fjellheisstasjon Narvik
Fitting 5 folds for each of 50 candidates, totalling 250 fits
Accuracy Score: 0.707
Training and evaluating model for location: Eikedalen Ski Center AS
Fitting 5 folds for each of 50 candidates, totalling 250 fits
Accuracy Score: 0.701
Training and evaluating model for location: Hemsedal Skisenter
Fitting 5 folds for each o